<a href="https://colab.research.google.com/github/usfb/npt_quot_gen_mk2/blob/main/NUPCO_Bid_Quotation_Generator_MVP%5Bworking_final_excel_input%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# ==============================================================================
# If running in Google Colab, you must first install LibreOffice for PDF conversion.
# Run this command in a separate Colab cell before executing this script:
# !sudo apt-get update && sudo apt-get install libreoffice
# ==============================================================================

# Cell 1: Imports and Setup

import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.drawing.image import Image
from datetime import datetime
import os
import re
import logging
import subprocess
import shutil
import math

def setup_logging():
    """Configures basic logging for the application."""
    # FIXED: Force logging configuration in Google Colab
    # Clear any existing handlers to avoid conflicts with Colab's default setup
    logger = logging.getLogger()
    logger.handlers.clear()

    # Set the root logger level
    logger.setLevel(logging.INFO)

    # Create a console handler with formatting
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Create formatter
    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(console_handler)

    # Force INFO level for this specific module
    logging.getLogger(__name__).setLevel(logging.WARNING)

    # Test logging to verify it's working
    logging.info("Logging setup complete - INFO messages should now be visible")

# --- Call logging setup immediately to ensure it's active ---
setup_logging()

# --- Helper function ---
def sanitize_filename(filename_str):
    """Sanitizes a string to be used as a filename."""
    filename_str = str(filename_str)
    return re.sub(r'[\\/:*?"<>|]', '_', filename_str).replace('/', '-').strip()

# Cell 2: Base Configuration & Styles
# ==============================================================================
# Base settings shared across all tender types
# ==============================================================================
# IMPORTANT: For Colab, ensure these files are uploaded to your environment.
FILE_PATHS = {
    'input_excel': '/content/worksheet_sample.xlsx',
    'header_image': '/content/header.jpg',
    'footer_image': '/content/footer.jpg',
    'stamp_image': '/content/stamp.jpg',
    'soffice_path': '/usr/bin/soffice' # Default path for LibreOffice in Colab/Linux
}

BASE_CONFIG = {
    'excel_output_dir': 'NUPCO_Quotations_Final',
    'pdf_output_dir': 'for_upload',
    'sheet_names': { 'input_sheet': 'Item', 'output_sheet': 'Quotation' },
    'quotation_defaults': { 'validity': '180 days', 'delivery': '90 days' },
    'layout': {
        'content_top_margin': 10,
        'footer_row_spacing': 15, # Spacing below the content table
    }
}

STYLES = {
    'default_font': Font(name='Calibri', size=11),
    'bold_font': Font(name='Calibri', size=11, bold=True),
    'missing_data': Font(name='Calibri', color="FF0000", bold=True),
    'align_left': Alignment(horizontal='left', vertical='center'),
    'align_center': Alignment(horizontal='center', vertical='center'),
    'align_wrap_center': Alignment(horizontal='center', vertical='center', wrap_text=True),
    'thin_border': Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin')),
    'header_fill': PatternFill(start_color="DDEBF7", end_color="DDEBF7", fill_type="solid"),
    'info_block_fill': PatternFill(start_color="EAF1FA", end_color="EAF1FA", fill_type="solid")
}

# Cell 3: Tender Type Configurations
# ==============================================================================
# Specific configurations for each tender type
# ==============================================================================

FRAMEWORK_CONFIG = {
    'tender_type': 'framework',
    'display_name': 'Framework Tender',
    'description': 'Traditional framework tender quotations with SRM codes',
    'mappings': {
        'input_columns': {
            'tender_number': 'Tender Number',
            'document_number': 'Document Number', # Added to mapping
            'item_number': 'Item Number',
            'material_description': 'Material Description',
            'catalog_number': 'Catalog Number',
            'material_number': 'Material Number',
            'srm_code': 'SRM CODE', # May be missing, logic will handle fallback
            'quantity': 'Quantity',
            'unit_price': 'Price',
            'vat_percentage': 'VAT %',
            'unit_of_measurement': 'Unit Of Measurement',
            'manufacturer_country': 'Manufacturer Country',
            'manufacturer': 'Manufacturer',
        },
        'output_cells': {
            'header_image_anchor': 'A1',
            'validity_label': {'row': 5, 'col': 2}, 'validity_value': {'row': 5, 'col': 3},
            'delivery_label': {'row': 6, 'col': 2}, 'delivery_value': {'row': 6, 'col': 3},
            'srm_code_label': {'row': 7, 'col': 2}, 'srm_code_value': {'row': 7, 'col': 3},
            'tender_no_label': {'row': 6, 'col': 6}, 'tender_no_value': {'row': 6, 'col': 7},
            'quot_no_label': {'row': 6, 'col': 8}, 'quot_no_value': {'row': 6, 'col': 9},
            'your_ref_label': {'row': 7, 'col': 6, 'text': 'Your Refr'}, 'your_ref_value': {'row': 7, 'col': 7},
            'date_label': {'row': 7, 'col': 8}, 'date_value': {'row': 7, 'col': 9},
        },
    },
    'table_settings': {
        'start_row': 9, 'start_col': 'B',
        'english_headers': [
            "Item", "Material Number", "Description", "Unit", "Qty", "Unit Price", "VAT Amount", "Total Price",
            "Manufacturer", "Made In", "P-code-xxx"
        ],
        'arabic_headers': [
            "", "رقم المادة", "البيـــــــــــــان", "الوحدة", "الكمية", "سعر الوحدة", "ض. ق. م.", "السعر الإجمالى",
            "الشركة المصنعة", "صنع في", ""
        ],
        'data_keys': [
            'item_number', 'material_number', 'material_description', 'unit_of_measurement', 'quantity',
            'unit_price', 'vat_amount', 'total_price_incl_vat', 'manufacturer', 'manufacturer_country',
            'p_code' # Unmapped key ensures P-code-xxx column returns "MISSING"
        ],
    },
    'layout': {
        'include_srm_code': True,
        'info_block_rows': 3,  # validity, delivery, srm_code
        'uses_document_number': True,  # Allow reading document number if present
    }
}

SUPPLIES_CONFIG = {
    'tender_type': 'supplies',
    'display_name': 'Supplies Tender',
    'description': 'Supplies tender quotations with document numbers and catalog numbers',
    'mappings': {
        'input_columns': {
            'tender_number': 'Tender Number',
            'item_number': 'Item Number',
            'material_description': 'Material Description',
            'catalog_number': 'Catalog Number',
            'material_number': 'Material Number',
            # 'srm_code': None,  # Not used in supplies tenders
            'quantity': 'Quantity',
            'unit_price': 'Price',
            'vat_percentage': 'VAT %',
            'unit_of_measurement': 'Unit Of Measurement',
            'manufacturer_country': 'Manufacturer Country',
            'manufacturer': 'Manufacturer',
            # Note: document_number will be handled separately from row 1
        },
        'output_cells': {
            'header_image_anchor': 'A1',
            'validity_label': {'row': 5, 'col': 2}, 'validity_value': {'row': 5, 'col': 3},
            'delivery_label': {'row': 6, 'col': 2}, 'delivery_value': {'row': 6, 'col': 3},
            # No SRM code row for supplies - rows shift up by 1
            'tender_no_label': {'row': 6, 'col': 6}, 'tender_no_value': {'row': 6, 'col': 7},
            'quot_no_label': {'row': 6, 'col': 8}, 'quot_no_value': {'row': 6, 'col': 9},
            'your_ref_label': {'row': 7, 'col': 6, 'text': 'Document Number'}, 'your_ref_value': {'row': 7, 'col': 7},
            'date_label': {'row': 7, 'col': 8}, 'date_value': {'row': 7, 'col': 9},
        },
    },
    'table_settings': {
        'start_row': 8, 'start_col': 'B',  # One row higher since no SRM code
        'english_headers': [
            "Item Number", "Description", "Unit", "Catalog Number", "Unit Price", "VAT Amount", "Total Price",
            "Manufacturer", "Made In", "Material Number"
        ],
        'arabic_headers': [
            "", "البيـــــــــــــان", "الوحدة", "رقم الكتالوج", "سعر الوحدة", "ض. ق. م.", "السعر الإجمالى",
            "الشركة المصنعة", "صنع في", "رقم المادة"
        ],
        'data_keys': [
            'item_number', 'material_description', 'unit_of_measurement', 'catalog_number',
            'unit_price', 'vat_amount', 'total_price_incl_vat', 'manufacturer', 'manufacturer_country',
            'material_number'
        ],
    },
    'layout': {
        'include_srm_code': False,
        'info_block_rows': 2,  # validity, delivery only
        'uses_document_number': True,  # Flag to indicate special document number handling
    }
}

# Registry of all available tender types
TENDER_TYPES = {
    'framework': FRAMEWORK_CONFIG,
    'supplies': SUPPLIES_CONFIG
}

def get_available_tender_types():
    """Returns list of available tender types with their display information."""
    return {key: {'display_name': config['display_name'], 'description': config['description']}
            for key, config in TENDER_TYPES.items()}

def get_tender_config(tender_type):
    """Returns the complete configuration for a specific tender type."""
    if tender_type not in TENDER_TYPES:
        raise ValueError(f"Unknown tender type: {tender_type}. Available types: {list(TENDER_TYPES.keys())}")

    # Merge base config with tender-specific config
    config = BASE_CONFIG.copy()
    tender_config = TENDER_TYPES[tender_type].copy()

    # Deep merge the configurations
    for key, value in tender_config.items():
        if isinstance(value, dict) and key in config:
            config[key].update(value)
        else:
            config[key] = value

    return config

def validate_tender_config(config):
    """Validates that the tender configuration has all required fields."""
    required_keys = ['tender_type', 'mappings', 'table_settings', 'layout']
    missing_keys = [key for key in required_keys if key not in config]

    if missing_keys:
        raise ValueError(f"Missing required configuration keys: {missing_keys}")

    # Validate mappings
    required_mappings = ['input_columns', 'output_cells']
    for mapping in required_mappings:
        if mapping not in config['mappings']:
            raise ValueError(f"Missing required mapping: {mapping}")

    logging.info(f"Configuration validation passed for tender type: {config['tender_type']}")
    return True

def select_tender_type():
    """Interactive function to select tender type."""
    available_types = get_available_tender_types()

    print("\n" + "="*60)
    print("NUPCO QUOTATION GENERATOR")
    print("="*60)
    print("\nAvailable Tender Types:")
    print("-" * 40)

    for i, (key, info) in enumerate(available_types.items(), 1):
        print(f"{i}. {info['display_name']}")
        print(f"   {info['description']}")
        print()

    while True:
        try:
            choice = input(f"Select tender type (1-{len(available_types)}): ").strip()
            choice_idx = int(choice) - 1

            if 0 <= choice_idx < len(available_types):
                selected_type = list(available_types.keys())[choice_idx]
                selected_info = available_types[selected_type]
                print(f"\nSelected: {selected_info['display_name']}")
                print("-" * 40)
                return selected_type
            else:
                print(f"Please enter a number between 1 and {len(available_types)}")

        except ValueError:
            print("Please enter a valid number")
        except KeyboardInterrupt:
            print("\nOperation cancelled by user")
            return None

def get_processing_options():
    """Interactive function to get processing options including test mode."""
    print("\nProcessing Options:")
    print("-" * 20)

    # Test mode selection
    while True:
        test_mode_input = input(
            "Enable test mode? (process only first 4 items) [y/N/number]: "
        ).strip().lower()

        if test_mode_input == '' or test_mode_input == 'n' or test_mode_input == 'no':
            test_limit = None
            print("✓ Processing ALL items in the file")
            break
        elif test_mode_input == 'y' or test_mode_input == 'yes':
            test_limit = 4
            print("✓ Test mode enabled: Processing first 4 items only")
            break
        else:
            # Check if user entered a custom number
            try:
                custom_limit = int(test_mode_input)
                if custom_limit > 0:
                    test_limit = custom_limit
                    print(f"✓ Custom test mode: Processing first {custom_limit} items only")
                    break
                else:
                    print("Please enter a positive number")
            except ValueError:
                print("Please enter 'y', 'n', or a number")

    return {
        'test_limit': test_limit,
        'is_test_mode': test_limit is not None
    }

def apply_test_limit(df, test_limit=None):
    """
    Applies test limit to dataframe if specified.

    Args:
        df: Input dataframe
        test_limit: Number of items to process (None for all items)

    Returns:
        Filtered dataframe and processing info
    """
    if test_limit is None:
        return df, {
            'total_available': len(df),
            'processing_count': len(df),
            'is_limited': False,
            'limit_applied': None
        }

    limited_df = df.head(test_limit)
    return limited_df, {
        'total_available': len(df),
        'processing_count': len(limited_df),
        'is_limited': True,
        'limit_applied': test_limit
    }

# Cell 4: Enhanced Core Logic Functions
# ==============================================================================
# Updated functions to handle multiple tender types
# ==============================================================================

def load_and_prepare_data(excel_path, config):
    """Enhanced data loading with tender-type specific handling."""
    logging.info(f"Loading data from '{os.path.basename(excel_path)}' for {config['tender_type']} tender...")

    try:
        # Read with header=1 to handle the unusual Excel structure
        df = pd.read_excel(excel_path, sheet_name=config['sheet_names']['input_sheet'], header=1)
        logging.info(f"Successfully read Excel file. Shape: {df.shape}")
        logging.info(f"Columns found: {list(df.columns)}")

        # For supplies tenders, read the Document Number from row 1, column B
        # Updated: Also for Framework tenders if we want to support hybrid files
        document_number = None
        if config.get('layout', {}).get('uses_document_number', False):
            try:
                # Read just the first row to get Document Number from B1
                header_df = pd.read_excel(excel_path, sheet_name=config['sheet_names']['input_sheet'],
                                        header=None, nrows=1)
                if len(header_df.columns) > 1:
                    document_number = header_df.iloc[0, 1]  # Row 1, Column B (0-indexed)
                    logging.info(f"Document Number from header (B1): {document_number}")
                else:
                    logging.warning("Could not find Document Number in expected location (Row 1, Column B)")
            except Exception as e:
                logging.warning(f"Could not read Document Number from header: {e}")

    except Exception as e:
        logging.error(f"Failed to load or parse Excel file: {e}")
        raise

    # Validate that required columns exist
    column_map = config['mappings']['input_columns']

    # Check specifically for SRM CODE for Framework tenders
    # NEW LOGIC: Just check if the SRM column exists. If not, we will hide the field in output.
    has_srm_column = False
    if config['tender_type'] == 'framework':
        srm_col = column_map.get('srm_code')
        if srm_col and srm_col in df.columns:
            has_srm_column = True
            logging.info(f"Found SRM CODE column: '{srm_col}'")
        else:
            logging.warning(f"SRM CODE column '{srm_col}' NOT found. The field will be hidden in output.")

    required_columns = [col for col in column_map.values() if col is not None]

    # If SRM code is missing, we don't treat it as a critical error anymore for Framework
    if config['tender_type'] == 'framework' and not has_srm_column:
        srm_key_val = column_map.get('srm_code')
        if srm_key_val in required_columns:
            required_columns.remove(srm_key_val)

    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        logging.error(f"Missing required columns in input file: {', '.join(missing_cols)}")
        raise ValueError(f"Missing required columns: {', '.join(missing_cols)}")

    # Filter rows based on 'Change Option' column - process only rows where it equals 'C'
    initial_row_count = len(df)
    filter_column = 'Change Option'

    if filter_column in df.columns:
        unique_values = df[filter_column].unique()
        logging.info(f"Unique values in '{filter_column}': {unique_values}")

        c_mask = df[filter_column] == 'C'
        c_count = c_mask.sum()

        logging.info(f"Found {c_count} rows with 'Change Option' == 'C' out of {initial_row_count} total rows")

        if c_count > 0:
            df = df[c_mask].copy()
            logging.info(f"Filtered for 'Change Option' == 'C'. Processing {len(df)} rows.")
        else:
            logging.error(f"No rows found with 'Change Option' == 'C'. Cannot proceed.")
            return None
    else:
        logging.error(f"Required filter column '{filter_column}' not found in the Excel file.")
        return None

    # Enhanced column mapping and validation
    processed_df = pd.DataFrame(index=df.index)

    # Store the srm availability flag in attrs
    processed_df.attrs['has_srm_column'] = has_srm_column

    for key, col_name in column_map.items():
        if col_name is None:
            continue

        if col_name in df.columns:
            processed_df[key] = df[col_name]
        else:
            processed_df[key] = pd.NA
            # Log warning for missing non-SRM columns (since we already logged SRM above)
            if key != 'srm_code':
                logging.warning(f"Column '{col_name}' not found, using NA for '{key}'")

    # Add document number logic
    # Priority: 1. Column in data (if mapped), 2. Header value (B1)
    if 'document_number' in processed_df.columns and not processed_df['document_number'].isna().all():
        logging.info("Using 'Document Number' from data columns.")
    elif document_number is not None:
        processed_df['document_number'] = document_number
        logging.info(f"Applied global document_number '{document_number}' to all rows.")

    # Ensure numeric conversion handles the data properly
    numeric_keys = ['quantity', 'unit_price', 'vat_percentage']
    for key in numeric_keys:
        if key in processed_df.columns:
            processed_df[key] = pd.to_numeric(processed_df[key], errors='coerce')

    # Calculate derived fields based on tender type
    if config['tender_type'] == 'supplies':
        # For supplies: Total = Unit Price + VAT Amount (no quantity multiplication)
        processed_df['subtotal'] = processed_df['unit_price'].fillna(0)  # Subtotal is just unit price
        processed_df['vat_amount'] = processed_df['subtotal'] * processed_df['vat_percentage'].fillna(0) / 100
        processed_df['total_price_incl_vat'] = processed_df['subtotal'] + processed_df['vat_amount']
    else:
        # For framework: Total = (Unit Price * Quantity) + VAT Amount
        processed_df['subtotal'] = processed_df['quantity'].fillna(0) * processed_df['unit_price'].fillna(0)
        processed_df['vat_amount'] = processed_df['subtotal'] * processed_df['vat_percentage'].fillna(0) / 100
        processed_df['total_price_incl_vat'] = processed_df['subtotal'] + processed_df['vat_amount']

    logging.info(f"Successfully prepared {len(processed_df)} items for {config['tender_type']} tender processing.")

    if len(processed_df) == 0:
        logging.error("No data rows to process after filtering and preparation!")
        return None

    return processed_df

def dynamically_adjust_layout(ws):
    """Dynamically adjusts column widths and row heights, then returns total content width in pixels."""
    column_widths = {}
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
        for cell in row:
            if cell.value:
                char_width_factor = 1.2
                cell_length = len(str(cell.value)) * char_width_factor
                if cell.alignment.wrap_text:
                    longest_word = max((len(word) for word in str(cell.value).split()), default=0)
                    cell_length = max(cell_length / 2, longest_word * char_width_factor)
                if cell.column not in column_widths or cell_length > column_widths[cell.column]:
                    column_widths[cell.column] = cell_length

    for col, max_length in column_widths.items():
        col_letter = get_column_letter(col)
        adjusted_width = min(max_length + 5, 60)
        ws.column_dimensions[col_letter].width = adjusted_width

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
        has_wrapped_text = any(cell.alignment.wrap_text and cell.value for cell in row)
        if has_wrapped_text:
            max_lines = 1
            for cell in row:
                if cell.alignment.wrap_text and cell.value:
                    col_width = ws.column_dimensions[get_column_letter(cell.column)].width
                    if col_width > 0:
                        lines = sum([math.ceil(len(part) * 1.2 / col_width) for part in str(cell.value).split('\n')])
                        if lines > max_lines: max_lines = lines
            new_height = (max_lines * 12) + 5
            ws.row_dimensions[row[0].row].height = new_height

    # Calculate total pixel width of content
    total_pixel_width = 0
    excel_unit_to_pixel = 7.5
    min_col, max_col = (99, 0)
    for row in ws.iter_rows():
        for cell in row:
            if cell.value:
                min_col = min(min_col, cell.column)
                max_col = max(max_col, cell.column)

    if max_col > 0:
        for col_idx in range(min_col, max_col + 1):
            col_letter = get_column_letter(col_idx)
            if ws.column_dimensions[col_letter].width:
                total_pixel_width += ws.column_dimensions[col_letter].width * excel_unit_to_pixel

    return total_pixel_width

def create_quotation_workbook(item_data, config, styles, file_paths):
    """Enhanced workbook creation with tender-type specific layouts."""
    FOOTER_ANCHOR_OFFSET = 9

    wb = Workbook()
    ws = wb.active
    ws.title = config['sheet_names']['output_sheet']

    ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
    ws.page_setup.paperSize = ws.PAPERSIZE_A4
    ws.page_setup.fitToPage = True
    ws.page_setup.fitToWidth = 1
    ws.page_setup.fitToHeight = 1

    ws.page_margins.left = 0; ws.page_margins.right = 0
    ws.page_margins.top = 0; ws.page_margins.bottom = 0

    item_num_for_logging = item_data.get('item_number', 'UNKNOWN')
    tender_type = config['tender_type']

    def get_data(key, default_val="MISSING"):
        val = item_data.get(key)
        if pd.isna(val) or val == '':
            logging.warning(f"Item '{item_num_for_logging}' ({tender_type}): Missing data for field '{key}'.")
            return default_val
        return val

    # Set default font for all cells
    for row in ws.iter_rows():
        for cell in row:
            cell.font = styles['default_font']

    cells = config['mappings']['output_cells']
    top_margin = config['layout']['content_top_margin']

    def set_cell(coord_key, value, font_style=None, align_style=None):
        coords = cells[coord_key]
        row_val = coords['row'] + top_margin
        cell = ws.cell(row=row_val, column=coords['col'], value=value)
        if font_style: cell.font = font_style
        if align_style: cell.alignment = align_style
        return cell

    # Set basic info cells
    set_cell('validity_label', "Quotation Validity", font_style=styles['bold_font'])
    set_cell('validity_value', config['quotation_defaults']['validity'])
    set_cell('delivery_label', "Delivery", font_style=styles['bold_font'])
    set_cell('delivery_value', config['quotation_defaults']['delivery'])

    # Conditionally add SRM code for framework tenders only
    if config['layout']['include_srm_code']:
        # We need to check if we should hide this.
        # Check if srm_code is valid data.
        srm_val = get_data('srm_code', 'MISSING')

        # If we passed a flag via item_data (hacky but works if we assign it to the series before iterating)
        # OR if we assume that if it returns 'MISSING', we check if we should hide it.
        # Let's assume for now: if column was missing, the value is NA.
        # We need to differentiate "Column Missing" from "Cell Empty".
        # I will check for a special attribute I will attach to item_data in the main loop.
        show_srm = True
        if hasattr(item_data, 'has_srm_column'):
             if not item_data.has_srm_column:
                 show_srm = False

        if show_srm:
            set_cell('srm_code_label', "SRM CODE", font_style=styles['bold_font'])
            set_cell('srm_code_value', srm_val)

    set_cell('tender_no_label', "Tender Number", font_style=styles['bold_font'])
    set_cell('tender_no_value', get_data('tender_number'))
    set_cell('quot_no_label', "Quotation Number", font_style=styles['bold_font'])
    set_cell('quot_no_value', f"{get_data('tender_number')}_{get_data('item_number')}")

    # Dynamic "Your Refr" vs "Document Number" logic
    # Default label from config
    ref_label = cells['your_ref_label']['text']
    ref_value = "MISSING"

    # Priority Logic for Reference Field
    doc_num = get_data('document_number', '')
    cat_num = get_data('catalog_number', '')

    if doc_num and str(doc_num) != 'MISSING':
        # If Document Number exists, use it and label accordingly
        ref_label = "Document Number"
        ref_value = doc_num
    else:
        # Fallback to Catalog Number (standard Framework behavior)
        ref_value = cat_num

    set_cell('your_ref_label', ref_label, font_style=styles['bold_font'])
    set_cell('your_ref_value', ref_value, align_style=styles['align_left'])

    set_cell('date_label', "Dated", font_style=styles['bold_font'])
    set_cell('date_value', datetime.now().strftime("%d/%m/%Y"))

    # Create borders for info block
    border_start_row = cells['tender_no_label']['row'] + top_margin
    border_end_row = cells['your_ref_label']['row'] + top_margin
    border_start_col = cells['tender_no_label']['col']
    border_end_col = cells['quot_no_value']['col']
    thin_side = Side(style='thin')

    for r in range(border_start_row, border_end_row + 1):
        for c in range(border_start_col, border_end_col + 1):
            cell = ws.cell(row=r, column=c)
            if c in [cells['tender_no_value']['col'], cells['your_ref_value']['col'],
                    cells['quot_no_value']['col'], cells['date_value']['col']]:
                cell.fill = styles['info_block_fill']
            top = thin_side if r == border_start_row else None
            bottom = thin_side if r == border_end_row else None
            left = thin_side if c in [cells['tender_no_label']['col'], cells['quot_no_label']['col']] else None
            right = thin_side if c in [cells['tender_no_value']['col'], cells['quot_no_value']['col']] else None
            cell.border = Border(top=top, bottom=bottom, left=left, right=right)

    # Create data table with tender-specific headers and data
    table_cfg = config['table_settings']
    english_header_row = table_cfg['start_row'] + top_margin
    arabic_header_row = english_header_row + 1
    data_row_idx = arabic_header_row + 1
    start_col_index = column_index_from_string(table_cfg['start_col'])

    # English headers
    for i, header_text in enumerate(table_cfg['english_headers']):
        cell = ws.cell(row=english_header_row, column=start_col_index + i, value=header_text)
        cell.font = styles['bold_font']
        cell.alignment = styles['align_wrap_center']
        cell.border = styles['thin_border']
        cell.fill = styles['header_fill']

    # Arabic headers
    for i, header_text in enumerate(table_cfg['arabic_headers']):
        cell = ws.cell(row=arabic_header_row, column=start_col_index + i, value=header_text)
        cell.font = styles['bold_font']
        cell.alignment = styles['align_wrap_center']
        cell.border = styles['thin_border']
        cell.fill = styles['header_fill']

    # Data row
    for i, data_key in enumerate(table_cfg['data_keys']):
        value = get_data(data_key)
        cell = ws.cell(row=data_row_idx, column=start_col_index + i, value=value)
        cell.border = styles['thin_border']
        cell.alignment = styles['align_wrap_center']
        if value == "MISSING":
            cell.font = styles['missing_data']

    # Dynamic layout & scaling
    content_pixel_width = dynamically_adjust_layout(ws) + 75

    # Add images with error handling
    try:
        if os.path.exists(file_paths['header_image']):
            header_img = Image(file_paths['header_image'])
            aspect_ratio = header_img.height / header_img.width if header_img.width > 0 else 0
            header_img.width = content_pixel_width - 50
            header_img.height = (content_pixel_width - 50) * aspect_ratio
            ws.add_image(header_img, cells['header_image_anchor'])
        else:
            logging.warning(f"Header image not found: '{file_paths['header_image']}'")
    except Exception as e:
        logging.warning(f"Could not load header image: {e}")

    footer_anchor_row = ws.max_row + config['layout']['footer_row_spacing'] + FOOTER_ANCHOR_OFFSET
    stamp_anchor_row = footer_anchor_row - 8

    try:
        if os.path.exists(file_paths['stamp_image']):
            stamp_img = Image(file_paths['stamp_image'])
            aspect_ratio = stamp_img.height / stamp_img.width if stamp_img.width > 0 else 0
            new_width = 225 # Scaled by 1.5x (150 * 1.5)
            stamp_img.width = new_width
            stamp_img.height = new_width * aspect_ratio
            ws.add_image(stamp_img, f'D{stamp_anchor_row}')
        else:
            logging.warning(f"Stamp image not found: '{file_paths['stamp_image']}'")
    except Exception as e:
        logging.warning(f"Could not load stamp image: {e}")

    try:
        if os.path.exists(file_paths['footer_image']):
            footer_img = Image(file_paths['footer_image'])
            aspect_ratio = footer_img.height / footer_img.width if footer_img.width > 0 else 0
            footer_img.width = content_pixel_width - 50
            footer_img.height = (content_pixel_width - 50) * aspect_ratio
            ws.add_image(footer_img, f'A{footer_anchor_row}')

            last_row_with_content = footer_anchor_row + math.ceil(footer_img.height / 15)
            ws.print_area = f'A1:{get_column_letter(ws.max_column+1)}{last_row_with_content}'
        else:
            logging.warning(f"Footer image not found: '{file_paths['footer_image']}'")
    except Exception as e:
        logging.warning(f"Could not load footer image: {e}")

    return wb

def convert_to_pdf(excel_path, pdf_dir, soffice_path):
    """Enhanced PDF conversion with better error handling."""
    logging.info(f"Converting '{os.path.basename(excel_path)}' to PDF...")
    try:
        if not os.path.exists(soffice_path):
            raise FileNotFoundError(f"LibreOffice not found at {soffice_path}")

        command = [soffice_path, '--headless', '--convert-to', 'pdf', '--outdir', pdf_dir, excel_path]
        result = subprocess.run(command, capture_output=True, text=True, check=True, timeout=60)
        logging.info(f"Successfully converted '{os.path.basename(excel_path)}'.")

        # Verify PDF was actually created
        pdf_filename = os.path.splitext(os.path.basename(excel_path))[0] + '.pdf'
        pdf_path = os.path.join(pdf_dir, pdf_filename)
        if not os.path.exists(pdf_path):
            logging.warning(f"PDF file was not created: {pdf_path}")

    except FileNotFoundError:
        logging.error(f"Error: LibreOffice not found at '{soffice_path}'. Please install LibreOffice.")
        raise
    except subprocess.TimeoutExpired:
        logging.error(f"PDF conversion timed out for '{os.path.basename(excel_path)}'")
        raise
    except subprocess.CalledProcessError as e:
        logging.error(f"LibreOffice conversion failed for '{os.path.basename(excel_path)}'. Return code: {e.returncode}")
        logging.error(f"STDOUT: {e.stdout}")
        logging.error(f"STDERR: {e.stderr}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error during PDF conversion: {e}")
        raise

# Cell 5: Enhanced Main Processing Functions
# ==============================================================================
# Updated main process with tender type selection and validation
# ==============================================================================

def run_full_process(tender_type, file_paths, test_limit=None):
    """
    Enhanced main process with tender type support and flexible test mode.

    Args:
        tender_type: Type of tender ('framework' or 'supplies')
        file_paths: Dictionary containing file paths
        test_limit: Number of items to process for testing (None for all items)

    Returns:
        dict: Processing results with detailed statistics
    """
    mode_desc = f"TEST MODE (first {test_limit} items)" if test_limit else "FULL PROCESSING"
    logging.info(f"--- Starting {mode_desc} for {tender_type.upper()} Tender ---")

    # Get and validate configuration
    try:
        config = get_tender_config(tender_type)
        validate_tender_config(config)
    except Exception as e:
        logging.error(f"Configuration error: {e}")
        return {'success': False, 'error': f"Configuration error: {e}"}

    # Validate input file exists
    if not os.path.exists(file_paths['input_excel']):
        error_msg = f"Input Excel file not found: {file_paths['input_excel']}"
        logging.error(error_msg)
        return {'success': False, 'error': error_msg}

    excel_dir = config['excel_output_dir']
    pdf_dir = config['pdf_output_dir']

    # Create output directories with tender type subdirectories
    excel_output_path = os.path.join(excel_dir, tender_type)
    pdf_output_path = os.path.join(pdf_dir, tender_type)

    os.makedirs(excel_output_path, exist_ok=True)
    os.makedirs(pdf_output_path, exist_ok=True)

    logging.info(f"Excel output will be saved to '{excel_output_path}'.")
    logging.info(f"PDF output will be saved to '{pdf_output_path}'.")

    try:
        # Load and prepare data
        df = load_and_prepare_data(file_paths['input_excel'], config)
        if df is None or len(df) == 0:
            error_msg = "No data to process after filtering."
            logging.error(error_msg)
            return {'success': False, 'error': error_msg}

        # Apply test limit if specified
        processed_df, limit_info = apply_test_limit(df, test_limit)

        # Propagate the has_srm_column flag from df to rows logic by attaching it to rows or handling in loop
        # Since iterrows returns Series which loses custom attrs, we grab the attr here
        has_srm_column = df.attrs.get('has_srm_column', True)

        # Log processing information
        if limit_info['is_limited']:
            logging.info(f"TEST MODE: Processing {limit_info['processing_count']} out of {limit_info['total_available']} available items")
        else:
            logging.info(f"FULL MODE: Processing all {limit_info['processing_count']} items")

        generated_count = 0
        failed_count = 0
        total_items = limit_info['processing_count']
        failed_items = []
        successful_items = []

        logging.info(f"Starting to process {total_items} items for {tender_type} tender...")

        for index, item_data in processed_df.iterrows():
            item_num = item_data.get('item_number', f'ROW_{index}')
            current_item = generated_count + failed_count + 1
            logging.info(f"Processing item {current_item}/{total_items} (Item Number: {item_num}) - {tender_type} tender...")

            # Manually attach the flag to the item_data Series object so create_quotation_workbook can see it
            item_data.has_srm_column = has_srm_column

            try:
                workbook = create_quotation_workbook(item_data, config, STYLES, file_paths)

                sanitized_item_num = sanitize_filename(str(item_num))
                excel_filename = f"{sanitized_item_num}quotation.xlsx"
                excel_filepath = os.path.join(excel_output_path, excel_filename)

                workbook.save(excel_filepath)
                logging.info(f"Saved Excel file: {excel_filename}")

                # PDF conversion
                try:
                    convert_to_pdf(excel_filepath, pdf_output_path, file_paths['soffice_path'])
                    generated_count += 1
                    successful_items.append({
                        'item_number': item_num,
                        'excel_file': excel_filename,
                        'pdf_file': excel_filename.replace('.xlsx', '.pdf')
                    })
                    logging.info(f"Successfully completed item {item_num}")
                except Exception as pdf_error:
                    logging.error(f"PDF conversion failed for item {item_num}: {pdf_error}")
                    failed_count += 1
                    failed_items.append({
                        'item_number': item_num,
                        'error': f"PDF conversion failed: {str(pdf_error)}",
                        'excel_file': excel_filename
                    })

            except Exception as e:
                logging.error(f"Failed to process item {item_num}. Error: {e}")
                failed_count += 1
                failed_items.append({
                    'item_number': item_num,
                    'error': str(e),
                    'excel_file': None
                })
                continue

        # Compile results
        results = {
            'success': True,
            'tender_type': tender_type,
            'processing_mode': 'test' if test_limit else 'full',
            'statistics': {
                'total_available': limit_info['total_available'],
                'total_processed': total_items,
                'successful': generated_count,
                'failed': failed_count,
                'success_rate': round((generated_count / total_items * 100), 2) if total_items > 0 else 0
            },
            'test_info': {
                'is_test_mode': limit_info['is_limited'],
                'test_limit': limit_info['limit_applied']
            },
            'output_paths': {
                'excel_dir': excel_output_path,
                'pdf_dir': pdf_output_path
            },
            'successful_items': successful_items,
            'failed_items': failed_items
        }

        # Log summary
        mode_text = f"TEST MODE ({test_limit} items)" if test_limit else "FULL PROCESSING"
        logging.info(f"--- {tender_type.upper()} Tender {mode_text} Complete ---")
        logging.info(f"Total available items: {limit_info['total_available']}")
        logging.info(f"Items processed: {total_items}")
        logging.info(f"Successfully completed: {generated_count}")
        logging.info(f"Failed: {failed_count}")
        logging.info(f"Success rate: {results['statistics']['success_rate']}%")

        return results

    except Exception as e:
        error_msg = f"Critical error during {tender_type} processing: {e}"
        logging.error(error_msg)
        return {'success': False, 'error': error_msg}

def test_setup(file_paths, tender_type=None):
    """Enhanced setup test with tender type validation."""
    logging.info("--- Testing Setup ---")

    issues = []

    # Check input file
    if not os.path.exists(file_paths['input_excel']):
        issues.append(f"Input Excel file not found: {file_paths['input_excel']}")
    else:
        logging.info("✓ Input Excel file found")

    # Check LibreOffice (optional for testing)
    if not os.path.exists(file_paths['soffice_path']):
        logging.warning(f"⚠ LibreOffice not found at: {file_paths['soffice_path']} (PDF conversion will fail)")
    else:
        logging.info("✓ LibreOffice found")

    # Check image files (optional)
    for img_type, img_path in [('header', file_paths['header_image']),
                               ('footer', file_paths['footer_image']),
                               ('stamp', file_paths['stamp_image'])]:
        if not os.path.exists(img_path):
            logging.warning(f"⚠ {img_type.title()} image not found: {img_path}")
        else:
            logging.info(f"✓ {img_type.title()} image found")

    # Test tender type configuration if specified
    if tender_type:
        try:
            config = get_tender_config(tender_type)
            validate_tender_config(config)
            logging.info(f"✓ {tender_type.title()} tender configuration validated")
        except Exception as e:
            issues.append(f"Tender configuration error for {tender_type}: {e}")

    # Test data loading for each available tender type or specific type
    test_types = [tender_type] if tender_type else list(TENDER_TYPES.keys())

    for test_type in test_types:
        try:
            config = get_tender_config(test_type)
            df = load_and_prepare_data(file_paths['input_excel'], config)
            if df is not None and len(df) > 0:
                logging.info(f"✓ Data loaded successfully for {test_type} tender. {len(df)} rows ready for processing.")
            else:
                issues.append(f"No data rows found after filtering for {test_type} tender")
        except Exception as e:
            issues.append(f"Failed to load data for {test_type} tender: {e}")

    # Only fail if there are critical issues (not missing images or LibreOffice)
    critical_issues = [issue for issue in issues if 'Input Excel file not found' in issue or 'configuration error' in issue or 'Failed to load data' in issue]

    if critical_issues:
        logging.error("Critical setup issues found:")
        for issue in critical_issues:
            logging.error(f"  - {issue}")
        return False
    else:
        if issues:
            logging.warning("Non-critical issues found (will continue with warnings):")
            for issue in issues:
                logging.warning(f"  - {issue}")
        logging.info("✓ Setup validation passed!")
        return True

def main():
    """Enhanced main function with interactive options and flexible processing."""
    print("NUPCO Quotation Generator - Modular Version")
    print("=" * 50)

    # Interactive tender type selection
    selected_tender_type = select_tender_type()

    if selected_tender_type is None:
        print("No tender type selected. Exiting.")
        return

    # Get processing options (including test mode)
    processing_options = get_processing_options()

    # Test setup for the selected tender type
    if test_setup(FILE_PATHS, selected_tender_type):
        # Run the process with selected options
        results = run_full_process(
            tender_type=selected_tender_type,
            file_paths=FILE_PATHS,
            test_limit=processing_options['test_limit']
        )

        # Display results
        display_processing_results(results)
    else:
        logging.error("Setup validation failed. Please fix the issues above before running.")
        print("\n❌ Setup validation failed. Check the logs above for details.")

def run_programmatic(tender_type, test_limit=None, file_paths=None):
    """
    Programmatic interface for running the quotation generator.

    This function can be called from other parts of a larger application.

    Args:
        tender_type: 'framework' or 'supplies'
        test_limit: Number of items to process (None for all)
        file_paths: Custom file paths (uses FILE_PATHS if None)

    Returns:
        dict: Detailed processing results
    """
    if file_paths is None:
        file_paths = FILE_PATHS

    # Validate tender type
    if tender_type not in TENDER_TYPES:
        return {
            'success': False,
            'error': f"Invalid tender type: {tender_type}. Available: {list(TENDER_TYPES.keys())}"
        }

    # Run setup test
    if not test_setup(file_paths, tender_type):
        return {
            'success': False,
            'error': "Setup validation failed"
        }

    # Execute processing
    return run_full_process(tender_type, file_paths, test_limit)

def display_processing_results(results):
    """Display formatted results of processing."""
    if not results['success']:
        print(f"\n❌ Processing failed: {results['error']}")
        return

    stats = results['statistics']
    tender_type = results['tender_type'].title()
    mode = results['processing_mode'].upper()

    print(f"\n✅ {tender_type} Tender Processing Complete ({mode} MODE)")
    print("=" * 60)

    if results['test_info']['is_test_mode']:
        print(f"📊 Test Mode Results (Limited to {results['test_info']['test_limit']} items):")
        print(f"   Total items available: {stats['total_available']}")
        print(f"   Items processed: {stats['total_processed']}")
    else:
        print(f"📊 Full Processing Results:")
        print(f"   Total items processed: {stats['total_processed']}")

    print(f"   ✅ Successful: {stats['successful']}")
    print(f"   ❌ Failed: {stats['failed']}")
    print(f"   📈 Success Rate: {stats['success_rate']}%")

    print(f"\n📁 Output Locations:")
    print(f"   Excel files: {results['output_paths']['excel_dir']}")
    print(f"   PDF files: {results['output_paths']['pdf_dir']}")

    # Show failed items if any
    if results['failed_items']:
        print(f"\n⚠️  Failed Items:")
        for item in results['failed_items'][:5]:  # Show first 5 failures
            print(f"   - Item {item['item_number']}: {item['error']}")
        if len(results['failed_items']) > 5:
            print(f"   ... and {len(results['failed_items']) - 5} more")

    # Test mode guidance
    if results['test_info']['is_test_mode'] and stats['successful'] > 0:
        print(f"\n💡 Test completed successfully! You can now run full processing.")
    elif not results['test_info']['is_test_mode']:
        print(f"\n🎉 Full processing completed!")

def quick_test(tender_type, num_items=4):
    """
    Quick test function for development.

    Args:
        tender_type: Type of tender to test
        num_items: Number of items to process (default: 4)

    Returns:
        dict: Processing results
    """
    print(f"\n🧪 Running quick test for {tender_type} tender ({num_items} items)...")
    return run_programmatic(tender_type, test_limit=num_items)

# Cell 6: Utility Functions for Development and Testing
# ==============================================================================
# Additional functions for debugging and development
# ==============================================================================

def preview_tender_differences():
    """Shows a comparison of the different tender types."""
    print("\n" + "="*80)
    print("TENDER TYPE COMPARISON")
    print("="*80)

    for tender_type, config in TENDER_TYPES.items():
        print(f"\n{config['display_name'].upper()}:")
        print("-" * 40)
        print(f"Description: {config['description']}")

        print(f"\nTable Headers:")
        for i, header in enumerate(config['table_settings']['english_headers']):
            print(f"  {i+1:2d}. {header}")

        print(f"\nSpecial Features:")
        if config['layout']['include_srm_code']:
            print("  - Includes SRM CODE field")
        else:
            print("  - No SRM CODE field")

        ref_field = config['mappings']['output_cells']['your_ref_label']['text']
        print(f"  - Reference field: '{ref_field}'")

        if config['layout']['uses_document_number']:
            print("  - Uses Document Number from Excel row 1")
        else:
            print("  - Uses Catalog Number from data columns")

        print()

def run_batch_test():
    """Runs a quick test for all tender types without full generation."""
    print("\n" + "="*60)
    print("BATCH TENDER TYPE TEST")
    print("="*60)

    for tender_type in TENDER_TYPES.keys():
        print(f"\nTesting {tender_type} tender configuration...")
        try:
            config = get_tender_config(tender_type)
            validate_tender_config(config)

            # Quick data load test
            df = load_and_prepare_data(FILE_PATHS['input_excel'], config)
            if df is not None and len(df) > 0:
                print(f"✅ {tender_type.title()}: Config OK, Data OK ({len(df)} rows)")
            else:
                print(f"⚠️  {tender_type.title()}: Config OK, No data found")
        except Exception as e:
            print(f"❌ {tender_type.title()}: Error - {e}")

def generate_sample_output(tender_type, item_index=0):
    """Generates a sample output for testing without full batch processing."""
    print(f"\n--- Generating Sample {tender_type.title()} Quotation ---")

    try:
        config = get_tender_config(tender_type)
        df = load_and_prepare_data(FILE_PATHS['input_excel'], config)

        if df is None or len(df) == 0:
            print("No data available for sample generation")
            return False

        if item_index >= len(df):
            print(f"Item index {item_index} out of range. Available: 0-{len(df)-1}")
            return False

        item_data = df.iloc[item_index]
        workbook = create_quotation_workbook(item_data, config, STYLES, FILE_PATHS)

        # Save sample
        sample_dir = f"sample_{tender_type}"
        os.makedirs(sample_dir, exist_ok=True)

        item_num = sanitize_filename(str(item_data.get('item_number', f'SAMPLE_{item_index}')))
        filename = f"{item_num}_quotation.xlsx"
        filepath = os.path.join(sample_dir, filename)

        workbook.save(filepath)
        print(f"✅ Sample saved: {filepath}")

        return True

    except Exception as e:
        print(f"❌ Sample generation failed: {e}")
        return False

# Cell 7: Entry Point
# ==============================================================================
# Main execution entry point
# ==============================================================================

if __name__ == '__main__':
    # Uncomment one of the following options:

    # Option 1: Run the full interactive process (recommended for normal use)
    main()

    # Option 2: Quick programmatic tests (for development)
    # results = quick_test('framework', 4)  # Test 4 framework items
    # results = quick_test('supplies', 4)   # Test 4 supplies items

    # Option 3: Programmatic full processing (for integration)
    # results = run_programmatic('framework')  # Full framework processing
    # results = run_programmatic('supplies', test_limit=10)  # Test 10 supplies items

    # Option 4: Preview tender type differences (for development)
    # preview_tender_differences()

    # Option 5: Run batch configuration test
    # run_batch_test()

    # Option 6: Generate samples for testing
    # generate_sample_output('framework', 0)  # Generate sample framework quotation
    # generate_sample_output('supplies', 0)   # Generate sample supplies quotation

2025-12-09 16:15:06 - INFO - Logging setup complete - INFO messages should now be visible


NUPCO Quotation Generator - Modular Version

NUPCO QUOTATION GENERATOR

Available Tender Types:
----------------------------------------
1. Framework Tender
   Traditional framework tender quotations with SRM codes

2. Supplies Tender
   Supplies tender quotations with document numbers and catalog numbers

Select tender type (1-2): 1

Selected: Framework Tender
----------------------------------------

Processing Options:
--------------------
Enable test mode? (process only first 4 items) [y/N/number]: 2


2025-12-09 16:15:10 - INFO - --- Testing Setup ---
2025-12-09 16:15:10 - INFO - ✓ Input Excel file found
2025-12-09 16:15:10 - INFO - ✓ LibreOffice found
2025-12-09 16:15:10 - INFO - ✓ Header image found
2025-12-09 16:15:10 - INFO - ✓ Footer image found
2025-12-09 16:15:10 - INFO - ✓ Stamp image found
2025-12-09 16:15:10 - INFO - Configuration validation passed for tender type: framework
2025-12-09 16:15:10 - INFO - ✓ Framework tender configuration validated
2025-12-09 16:15:10 - INFO - Loading data from 'worksheet_sample.xlsx' for framework tender...


✓ Custom test mode: Processing first 2 items only


2025-12-09 16:15:10 - INFO - Successfully read Excel file. Shape: (694, 30)
2025-12-09 16:15:10 - INFO - Columns found: ['Tender Number', 'Document Number', 'Change Option', 'Item Type', 'Item Number', 'Material Number', 'Material Description', 'Location', 'Equipment Group', 'Scoring', 'Quantity', 'Unit Of Measurement', 'Price', 'Currency', 'VAT %', 'Manufacturer', 'Manufacturer Country', 'SFDA', 'Model ID:', 'Catalog Number', 'Item Validity', 'Package Size', 'Official Agent', 'Company Comments', 'Company Comment Term', '1st Lead time delive', 'Lead time delivery p', 'Max number of shipme', 'Embeded Type', 'FOC Quantity']
2025-12-09 16:15:11 - INFO - Document Number from header (B1): Document Number
2025-12-09 16:15:11 - WARNING - SRM CODE column 'SRM CODE' NOT found. The field will be hidden in output.
2025-12-09 16:15:11 - INFO - Unique values in 'Change Option': ['C' nan]
2025-12-09 16:15:11 - INFO - Found 235 rows with 'Change Option' == 'C' out of 694 total rows
2025-12-09 16:15:1


✅ Framework Tender Processing Complete (TEST MODE)
📊 Test Mode Results (Limited to 2 items):
   Total items available: 235
   Items processed: 2
   ✅ Successful: 2
   ❌ Failed: 0
   📈 Success Rate: 100.0%

📁 Output Locations:
   Excel files: NUPCO_Quotations_Final/framework
   PDF files: for_upload/framework

💡 Test completed successfully! You can now run full processing.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')